In [27]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math
import cmath

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

# Our imports
import skimage
import scipy
from skimage import util,color,exposure,io
from scipy import fftpack

%matplotlib inline

In [98]:
def FT(im,u,v,imaginary):
    s = complex(0,0)
    
    M,N = im.shape
    
    for x in range(M):
        for y in range(N):
            s += im[x,y] * cmath.exp(imaginary * 2 * cmath.pi * (((u*x) / M) + ((v*y) / N)))
    return s


def DFT(im):
    height, width = im.shape
    
    out = numpy.zeros(im.shape,dtype=numpy.complex)
    
    for u in range(height):
        for v in range(width):
            out[u,v] = FT(im,u,v,-1j)
    
    return out

def IDFT(im):
    height, width = im.shape
    
    out = numpy.zeros(im.shape,dtype=numpy.complex)
    
    for u in range(height):
        for v in range(width):
            out[u,v] = 1/(height*width) * FT(im,u,v,1j)
    
    return out

In [112]:
def fourrier(image):
    # Read image, convert to gray, convert to float
    im = numpy.asarray([[0.7, 1, 0, 0.5], [0, 0, 1, 0], [0.3, 1, 0, 0.5], [0, 0, 1, 0]], dtype=numpy.float)

    # Retrieve the image dimensions
    height,width = im.shape

    # Create an image to store the transformed image
    image_trans = numpy.zeros(im.shape)
    # Transform each pixel in the image by multiplying with -1**(i+j) (see 
    # slides/book) to translate the fourier transformed image to height/2 width/2 
    # i.e. the center of the image
    for i in range(height):
      for j in range(width):
        image_trans[i,j] = im[i,j]*((-1)**(i+j))

    # Perform the fourier transformation
    ft_builtin = fftpack.fft2(image_trans)
    ft = DFT(image_trans)

    # Get the fourier spectrum by finding the length of the complex number vectors
    ft_abs = numpy.absolute(ft)
    ft_builtin_abs = numpy.absolute(ft_builtin)

    # Log transformation (see own lecture)
    out = numpy.zeros(im.shape)
    out2 = numpy.zeros(im.shape)
    for i in range(0, im.shape[0]):
      for j in range(0, im.shape[1]):
        out[i,j] = numpy.log(1+ft_abs[i,j])
        out2[i,j] = numpy.log(1+ft_builtin_abs[i,j])
        
     #transformerer med invers fourier
    filtered = fftpack.ifft2(ft_builtin)
    
    #translerer tilbake
    filtered_translated = numpy.zeros(im.shape)

    for i in range(height):
        for j in range(width):
            filtered_translated[i,j] = filtered[i,j]*((-1)**(i+j))


    # Show the result
    io.imshow_collection([out,out2, numpy.rea(IDFT(ft)),filtered_translated], cmap='gray')
    io.show()
    
interactive(fourrier,image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'hurricane': '../Bilder/image_resources/hurrica…